<a href="https://colab.research.google.com/github/econ105/AI/blob/main/SectorswiththebestreturninHKstock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#Requirements:
# pip install yfinance pandas numpy openpyxl

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# Hong Kong Hang Seng sectors with representative tickers/stocks
hk_sectors = {
    'Finance': ['0005.HK', '0011.HK', '0388.HK', '1299.HK', '2318.HK'],  # HSBC, Hang Seng Bank, HKEx, AIA, Ping An
    'Utilities': ['0002.HK', '0003.HK', '0006.HK', '0836.HK'],           # CLP, HK-China Gas, Power Assets, China Res Power
    'Properties': ['0012.HK', '0016.HK', '0017.HK', '0823.HK', '0688.HK'], # Henderson Land, Sun Hung Kai, New World, Link REIT, China Overseas
    'Commerce_Industry': ['0700.HK', '0998.HK', '3690.HK', '1810.HK', '1211.HK']  # Tencent, Alibaba, Meituan, Xiaomi, BYD
}

print("📥 Downloading Hong Kong sector returns data...")
print("=" * 60)

# Flatten all tickers
all_tickers = []
sector_mapping = {}
for sector, tickers in hk_sectors.items():
    all_tickers.extend(tickers)
    for ticker in tickers:
        sector_mapping[ticker] = sector

# FIXED: Remove group_by and use Close (auto_adjust=True makes Close = Adj Close)
start_date = "2015-01-01"
data = yf.download(all_tickers, start=start_date, progress=False)  # No group_by!
prices = data['Close'].dropna(axis=1, how='all')  # Use Close directly

print(f"Downloaded data for {len(prices.columns)} stocks across 4 sectors")

# Calculate returns
daily_returns = prices.pct_change().dropna()
cumulative_returns = (1 + daily_returns).cumprod()

# Sector performance summary
trading_days_per_year = 252
sector_summary = {}

for sector, tickers in hk_sectors.items():
    sector_data = prices[[t for t in tickers if t in prices.columns]]
    if len(sector_data.columns) > 0:
        sector_ret = sector_data.pct_change().dropna()
        total_return = sector_data.iloc[-1] / sector_data.iloc[0] - 1
        ann_return = (1 + total_return) ** (trading_days_per_year / len(sector_ret)) - 1
        ann_vol = sector_ret.std() * np.sqrt(trading_days_per_year)

        sector_summary[sector] = {
            'Stocks': len(sector_data.columns),
            'Total Return': total_return.mean(),
            'Ann Return': ann_return.mean(),
            'Ann Volatility': ann_vol.mean()
        }

# Create summary DataFrame
summary_df = pd.DataFrame(sector_summary).T.round(4)
summary_df = summary_df.sort_values('Ann Return', ascending=False)

# Export to Excel
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
excel_file = f"HK_Sectors_Returns_{timestamp}.xlsx"

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    prices.to_excel(writer, sheet_name='Daily_Prices')
    daily_returns.to_excel(writer, sheet_name='Daily_Returns')
    cumulative_returns.to_excel(writer, sheet_name='Cumulative_Returns')
    summary_df.to_excel(writer, sheet_name='Sector_Summary')

print(f"\n💾 Excel saved: {excel_file}")
print("\n📊 Sheets:")
print("• Daily_Prices - Individual stock prices")
print("• Daily_Returns - Individual stock daily returns")
print("• Cumulative_Returns - Growth of HK$1")
print("• Sector_Summary - Sector performance comparison")

print("\n🏆 Sector Performance Ranking:")
print(summary_df[['Ann Return', 'Ann Volatility']])


📥 Downloading Hong Kong sector returns data...


/tmp/ipython-input-736976979.py:30: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, start=start_date, progress=False)  # No group_by!


Downloaded data for 19 stocks across 4 sectors

💾 Excel saved: HK_Sectors_Returns_20260125_0857.xlsx

📊 Sheets:
• Daily_Prices - Individual stock prices
• Daily_Returns - Individual stock daily returns
• Cumulative_Returns - Growth of HK$1
• Sector_Summary - Sector performance comparison

🏆 Sector Performance Ranking:
                   Ann Return  Ann Volatility
Commerce_Industry      0.2615          0.4273
Finance                0.0922          0.2805
Utilities              0.0392          0.2173
Properties             0.0054          0.2798


In [3]:
# Requirements:
# pip install yfinance pandas numpy openpyxl

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# Hong Kong Hang Seng sectors with major constituent companies
hk_sectors = {
    'Finance': {
        '0005.HK': 'HSBC Holdings',
        '0011.HK': 'Hang Seng Bank',
        '0388.HK': 'Hong Kong Exchanges',
        '1299.HK': 'AIA Group',
        '2318.HK': 'Ping An Insurance'
    },
    'Utilities': {
        '0002.HK': 'CLP Holdings',
        '0006.HK': 'Power Assets',
        '0003.HK': 'Hong Kong & China Gas',
        '0836.HK': 'China Resources Power'
    },
    'Properties': {
        '0016.HK': 'Sun Hung Kai Properties',
        '0012.HK': 'Henderson Land',
        '0017.HK': 'New World Development',
        '0823.HK': 'Link REIT',
        '0688.HK': 'China Overseas Land'
    },
    'Commerce_Industry': {
        '0700.HK': 'Tencent Holdings',
        '0998.HK': 'Alibaba Group',
        '3690.HK': 'Meituan',
        '1810.HK': 'Xiaomi Corp',
        '1211.HK': 'BYD Company'
    }
}

print("📊 Individual Company Returns & Volatility Analysis")
print("=" * 80)

# Download data for all companies
start_date = "2015-01-01"
all_tickers = []
for sector, companies in hk_sectors.items():
    all_tickers.extend(companies.keys())

print(f"Downloading data for {len(all_tickers)} companies...")

data = yf.download(all_tickers, start=start_date, progress=False)
prices = data['Close'].dropna(axis=1, how='all')

# Calculate metrics for each company
trading_days_per_year = 252
company_results = []

for ticker in prices.columns:
    stock_prices = prices[ticker].dropna()
    if len(stock_prices) > 100:  # Ensure sufficient data

        daily_returns = stock_prices.pct_change().dropna()

        # Key metrics
        total_return = (stock_prices.iloc[-1] / stock_prices.iloc[0]) - 1
        ann_return = (1 + total_return) ** (trading_days_per_year / len(daily_returns)) - 1
        ann_volatility = daily_returns.std() * np.sqrt(trading_days_per_year)
        sharpe_ratio = ann_return / ann_volatility if ann_volatility > 0 else 0

        # Find company name and sector
        company_name = next((name for s, companies in hk_sectors.items()
                           for t, name in companies.items() if t == ticker), ticker)
        sector = next((s for s, companies in hk_sectors.items()
                      if ticker in companies), 'Unknown')

        company_results.append({
            'Sector': sector,
            'Ticker': ticker,
            'Company': company_name,
            'Total Return': total_return,
            'Annualized Return (%)': ann_return * 100,
            'Annualized Volatility (%)': ann_volatility * 100,
            'Sharpe Ratio': sharpe_ratio,
            'Data Points': len(daily_returns)
        })

# Create results DataFrame
results_df = pd.DataFrame(company_results).round(4)

# Sector averages
sector_averages = results_df.groupby('Sector').agg({
    'Annualized Return (%)': 'mean',
    'Annualized Volatility (%)': 'mean',
    'Sharpe Ratio': 'mean',
    'Ticker': 'count'
}).round(4)
sector_averages.columns = ['Avg Return (%)', 'Avg Volatility (%)', 'Avg Sharpe', 'Companies']
sector_averages = sector_averages.sort_values('Avg Return (%)', ascending=False)

# Export to Excel
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
excel_file = f"HK_Sector_Companies_Returns_{timestamp}.xlsx"

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    # Detailed company analysis
    results_df.sort_values('Annualized Return (%)', ascending=False).to_excel(
        writer, sheet_name='Company_Details', index=False)

    # Sector summary
    sector_averages.to_excel(writer, sheet_name='Sector_Averages')

    # Raw prices
    prices.to_excel(writer, sheet_name='Raw_Prices')

print(f"\n💾 Excel saved: {excel_file}")
print("\n📊 RESULTS SUMMARY:")

print("\n🏆 TOP 10 COMPANIES by Annualized Return:")
top_companies = results_df.nlargest(10, 'Annualized Return (%)')
print(top_companies[['Company', 'Sector', 'Annualized Return (%)', 'Annualized Volatility (%)']])

print("\n📈 SECTOR AVERAGES:")
print(sector_averages[['Avg Return (%)', 'Avg Volatility (%)', 'Companies']])

print(f"\n✅ Analyzed {len(results_df)} companies across {len(sector_averages)} sectors")


📊 Individual Company Returns & Volatility Analysis


/tmp/ipython-input-305741447.py:51: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, start=start_date, progress=False)



💾 Excel saved: HK_Sector_Companies_Returns_20260125_0859.xlsx

📊 RESULTS SUMMARY:

🏆 TOP 10 COMPANIES by Annualized Return:
                Company             Sector  Annualized Return (%)  \
14          BYD Company  Commerce_Industry                23.6700   
10     Tencent Holdings  Commerce_Industry                18.0432   
2         HSBC Holdings            Finance                11.4664   
8   Hong Kong Exchanges            Finance                11.4511   
16          Xiaomi Corp  Commerce_Industry                10.9900   
17    Ping An Insurance            Finance                 8.5566   
15            AIA Group            Finance                 8.0486   
13        Alibaba Group  Commerce_Industry                 7.9237   
4        Hang Seng Bank            Finance                 6.5883   
3          Power Assets          Utilities                 5.6453   

    Annualized Volatility (%)  
14                    48.1790  
10                    34.8684  
2                  